## Title Basics

Exploring and Profiling data

## Cleaning table - transformation and standardisation
- standardise field names and types
- replace rogue calues - isAdult
- enrich and convert to arrays - genres


In [0]:
WITH

-- Clean base table
cleaned_base AS(
SELECT 

    LOWER(TRIM(tconst)) AS id_number,        -- Keep 'tt' prefix, as it is the IMDb key
    titleType AS title_type,
    primaryTitle AS primary_title,
    originalTitle AS original_title,
    startYear AS start_year,
    
    -- Replace values NOT IN (0,1) to false for isAdult, then convert to boolean
    CASE
      WHEN isAdult NOT IN (0,1) THEN 'false'
      ELSE CAST(isAdult AS BOOLEAN)
      END AS is_adult,

    -- NOT REQUIRED: "\NN" Have been automatically nullfied
    --Replace '\N' with NULL and cast to INT for endYear and runtimeMinutesCASE WHEN endYear = '\N' THEN NULL ELSE CAST(endYear AS INT) END AS end_year, CASE  WHEN runtimeMinutes = '\N' THEN NULL ELSE CAST(runtimeMinutes AS INT) END AS runtime_minutes,

    endYear AS end_year,
    runtimeMinutes AS runtime_minutes,
    genres AS genres


FROM imdb.raw.title_basics
),

-- Fix column misalignment: primary_title and original_title
  mislaignment_clean AS(
    SELECT * EXCEPT(primary_title, original_title),

      -- fixing for primary_title
      CASE WHEN primary_title LIKE '"%\t"%' 
        THEN substring(array_distinct(SPLIT(primary_title, '\t'))[0], 2)   
        /*01. split merged string into array
          02. array_distinct to remove duplicate array
          03. cast to string type by selecting 0th element in distinct array
          04. substring(___, 2) starts the string on the second cahracter, thus removing the bginining quotation mark */
      ELSE primary_title
      END AS primary_title,

      --fixing for original_title
      CASE WHEN primary_title LIKE '"%\t"%' 
        THEN substring(array_distinct(SPLIT(primary_title, '\t'))[0], 2)   -- this is just duplicate of cleaned primary_title
      ELSE original_title
      END AS original_title


    FROM cleaned_base
  ),

-- Enrich genres field and convert to array
  --Enrich Logic: if isAdult = 1 but genres does not have "Adult", then add "Adult" in genres
  enrich_and_convert_genres AS(
    SELECT * EXCEPT(genres),
      CASE WHEN
        is_adult = 'true' AND LOWER(genres) NOT LIKE "%adult%"
          THEN SPLIT(TRIM(LOWER(CONCAT(genres, ',Adult'))), ',')
        ELSE SPLIT(TRIM(LOWER(genres)), ',')
        END AS genres
    FROM mislaignment_clean
  )




-- Option 2 (genres conversion to arrays): Exploding and repacking
/*
,
  -- Exploding genres to helper column
  exploded AS (
    SELECT * 
    FROM cleaned_base
  LATERAL VIEW EXPLODE(SPLIT(TRIM(genres),','))
      AS genres_exploded
      WHERE genres_exploded IS NOT NULL
  ),                                                    -- this steps creates mutliple rows for each id_number based on how many unique genres there are

  -- Repack genres_exploded into arrays
  repacked AS(
    SELECT * EXCEPT (genres, genres_exploded),
  -- repack: groups data by id_number etc to collect exploded genres into single arrays
      COLLECT_SET(genres_exploded) AS genres
    FROM exploded
    GROUP BY id_number, title_type, primary_title, original_title, start_year, is_adult, end_year, runtime_minutes
  )
*/

--SELECT * FROM cleaned_base WHERE primary_title LIKE '"%\t"%'
SELECT 
  id_number,
  title_type,
  primary_title,
  original_title,
  is_adult,
  start_year,
  end_year,
  runtime_minutes,
  genres
 FROM enrich_and_convert_genres;


## Data Validation Checks

1) Data types check
2) Row count check
3) Identify functional types (i.e. Keys, Dimensions, Measures)
4) Data quality checks + DQ Audits

In [0]:
-- Data types check
DESCRIBE imdb.raw.title_basics;

In [0]:
-- Row count check
SELECT COUNT(*),
  LEN(COUNT(*)) AS digits
 FROM imdb.raw.title_basics;

In [0]:

-- Data Dictionary

SELECT * FROM (
  VALUES 
    ('tconst', 'Alphanumeric unique identifier of the title', 'Identifier'),
    ('titleType', 'Format of the title (e.g., movie, tvSeries)', 'Dimension'),
    ('primaryTitle', 'The popular/promotional title at release', 'Dimension'),
    ('originalTitle', 'Original title in the original language', 'Dimension'),
    ('isAdult', 'Boolean flag: 0=non-adult, 1=adult', 'Dimension'),
    ('startYear', 'Release year or series start year (YYYY)', 'Measure'),
    ('endYear', 'Series end year if applicable (YYYY)', 'Measure'),
    ('runtimeMinutes', 'Primary runtime in minutes', 'Measure'),
    ('genres', 'Array of up to three genres', 'Dimension')
) AS data_dictionary(field_name, description, functional_type);

**Data Quality (DQ) Checks - column by column**

Checks:  
1) Completeness: checks involving null values  
2) Distinctiveness: checks involving duplicates
3) Uniformity: checks involving outliers, rogue values and redundant logic

DQ Check - tconst (id_number)

In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE tconst IS NULL
  ), -- no null values in tconst

-- Distinctiveness
  duplicate_check_tconst AS(
    SELECT tconst, COUNT(*)
    FROM base_table
    GROUP BY tconst
    HAVING COUNT(*) > 1
  ),  -- no duplicates in tconst


-- Uniformity 
  -- no outliers detected

SELECT * FROM  duplicate_check_tconst


DQ CHECK - titleType (title_type)


In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE titleType IS NULL
  ), -- no null values in tconst

-- Distinctiveness
  -- no checks necessary


-- Uniformity 
  distinct_titleType AS(
    SELECT DISTINCT titleType FROM base_table
  )
  -- no outliers detected

SELECT * FROM distinct_titleType


DQ CHECK - primaryTitle (primary_title)

In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE primaryTitle IS NULL
  ), -- no null values in tconst

-- Distinctiveness
  -- no more checks

-- Uniformity 
  pipe_delimiter_check AS (
    SELECT * FROM imdb.raw.title_basics 
      WHERE primaryTitle LIKE "%|%"
  )     -- many entries with pipe delimiter, dq audit required

SELECT * FROM pipe_delimiter_check


In [0]:
-- DQ Audit - primaryTitle (pipe delimiter)
  -- entries with pipe delimiters seem to correspond to YOUTUBE videos

WITH
pipe_delimiter_proportion AS(
  SELECT 
    COUNT(*) AS pipe_delimiter_count,
    (SELECT COUNT(*) FROM imdb.raw.title_basics) AS total_count,
    (COUNT(*) / (SELECT COUNT(*) FROM imdb.raw.title_basics) ) AS pipe_delimiter_proportion
  FROM imdb.raw.title_basics 
    WHERE primaryTitle LIKE "%|%"
),

-- Check whether these entries correspond to specific titleType
titleType_correspondence AS(
  SELECT DISTINCT titleType
  FROM imdb.raw.title_basics
  WHERE primaryTitle LIKE "%|%"
) -- no specific titleType associated with youtube videos
-- assign "webContent" to titleType for entries that include "%|%"

SELECT * FROM titleType_check



DQ CHECK - originalTitle (original_title)
Notes:
- column misalignment: originalTitle merged into primaryTitle, resulting in '0' output


In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE originalTitle IS NULL
  ), -- no null values

-- Distinctiveness
  duplicate_check_originalTitle AS(
    SELECT originalTitle, COUNT(*)
    FROM base_table
    GROUP BY 1
    HAVING COUNT(*) > 1
  ), -- duplicate values exist, but do not delete due to unique tconst

  -- Checking how many entries have unique originalTitle and primaryTitle
    non_duplicate_check_primaryTitle_originalTitle AS(
    SELECT * 
    FROM base_table
    WHERE primaryTitle <> originalTitle
    
  ), -- difference in primary and original title are most commonly translations into english (originalTitle = non-english, primary = english)
  
  non_duplicate_count_primaryTitle_orginalTitle AS(
    SELECT 
    COUNT(*) AS non_duplicate_count,
    (SELECT COUNT(*) FROM base_table) AS total_count,
    COUNT(*) / (SELECT COUNT(*) FROM base_table) AS proportion
    FROM base_table
    WHERE primaryTitle <> originalTitle
  ),  -- only small portion of entries have different primary and original title
        -- most originalTitle are primaryTitle

  

-- Uniformity 
  rouge_value_check AS(
    SELECT * 
    FROM base_table  
      WHERE originalTitle = '0' 
      OR originalTitle = '1'
  ) -- column misalignment present, originalTitle seems to have shifted to primaryTitle (e.g."Art Brings People Close	"Art Brings People Close)
      -- requires DQ audit

SELECT * FROM  rouge_value_cehck



In [0]:
-- DQ Audit: originalTitle (column misalignment)
WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
),

-- identify misalignment pattern
debug_condition_1 AS(
  SELECT * FROM base_table
    -- identify pattern ("____  "____), where the middle gap is a 'tab' not regular 'space'
    WHERE primaryTitle LIKE '"%	"%'
  ),

-- checking against intial rogue values identified
rogue_value_cross_check AS(
  SELECT * 
  FROM base_table a
      LEFT ANTI JOIN debug_condition_1 b ON a.primaryTitle = b.primaryTitle
    WHERE originalTitle = '0' 
    OR originalTitle = '1'
  ), -- many rogue values are acutally accurate (e.g. title actually is '1')

-- confirm whether the misaligned primaryTitle are duplicates
duplicate_pattern_check AS(
  SELECT COUNT(*)
  FROM debug_condition_1
    -- size of array =  1 if the misaligned primaryTitle have duplicates (primaryTitle and originalTitle merged into one merged string"
    WHERE SIZE(array_distinct(SPLIT(primaryTitle,'  '))) > 1
) -- pattern confirmed

SELECT * FROM duplicate_pattern_check

  -- transformations required to remove misalignment and populate the originalTitle field
    -- note, this misalignment also led to isAdult values NOT IN (0, 1), which will be investigated in below cells

DQ CHECK - isAdult (is_adult)

Notes: 
1) Add "Adult" classification to genres field for all isAdult = 1 entries
2) Assume isAdult = 1 when encountering values NOT IN (0, 1)


In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE isAdult IS NULL
  ), -- null values exist

-- Distinctiveness
  -- no checks necessary

-- Uniformity 
  distinct_values_check AS(
    SELECT DISTINCT isAdult
    FROM base_table
  )   -- many distinct values, should only be either 0 or 1
          -- requires further DQ Audit



SELECT * FROM distinct_values_check


In [0]:
-- DQ Audit - isAdult (rogue values found)
  -- isAdult is not boolean

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

initial_check AS(
  SELECT * FROM base_table
  WHERE isAdult NOT IN (0, 1)
), -- non-boolean values seem to occur when originalTitle = 0 or originalTitle = 1

  isAdult_originalTitle_relationship AS(
    SELECT COUNT(*) 
    FROM base_table
    WHERE isAdult NOT IN (0, 1) 
    AND originalTitle NOT IN ("0", "1")
  ), -- confirmed above relationship with non-boolean values and originalTitle

-- Cross-reference: isAdult gnere
  -- 1) Does all isAdult = 1 whern "Adult" in genres field?
    cross_reference_isAdult_genres_1 AS(
      SELECT COUNT(*) AS inverse_conditional_count,
        (SELECT COUNT(*) FROM base_table) AS total_count,
        (SELECT COUNT(*) / (SELECT COUNT(*) FROM base_table)) AS proportion
      FROM base_table
      WHERE isAdult = 1
      AND TRIM(LOWER(genres)) NOT LIKE "%adult%"
    ), -- condition holds true in almost all cases (8e-6% exception)
        -- assume all isAdult = 1 entries have "Adult" in genres field, add these entries during cleaning
  
  -- 2) How many isAdult <> (0, 1) have "Adult" in genres field?
    cross_reference_isAdult_genres_2 AS(
      SELECT COUNT(*)
      FROM base_table
      WHERE isAdult NOT IN (0, 1) 
      AND TRIM(LOWER(genres)) LIKE "%adult%"
    ) -- 0 entries

SELECT * FROM cross_reference_isAdult_genres_2

-- Cleaning to be made
  -- 1) Add "Adult" classification to genres field for all isAdult = 1 entries
  -- 2) Assume isAdult = 1 when encountering values NOT IN (0, 1)

DQ CHECK - startYear/endYear (start_year / end_year)

In [0]:
WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

  -- Completeness
  null_proportions AS(
    SELECT COUNT (*) AS startYear_null_count,
    (SELECT COUNT(*) FROM base_table WHERE endYear IS NULL) AS endYear_null_count,
    (SELECT COUNT(*) FROM base_table) AS total_count,
    (COUNT(*) / (SELECT COUNT(*) FROM base_table)) AS startYear_null_proportion,
    ((SELECT COUNT(*) FROM base_table WHERE endYear IS NULL) / (SELECT COUNT(*) FROM base_table)) AS endYear_null_proportion
      FROM base_table
      WHERE startYear IS NULL
  ),
    -- startYear: 12% null
    -- endYear: 99% null

  -- Uniformity

  -- Checking whether there are start or end years are future dates
  futuredateyears_check AS(
    SELECT * FROM base_table
    WHERE 
      startYear > YEAR(CURRENT_DATE)
      OR
      endYear > YEAR(CURRENT_DATE)  --7000+ imdb entries that are upcoming or ending in the future

    -- add is_upcoming flag in staging -> transformed?
  )

  SELECT * FROM upcoming_check

DQ CHECK - genres (genres)  

Notes:
1) convert merged strings into arrays using split(field, ',')
2) add 'Adult' to genres field for all isAdult = 1 entries (if not exist already)


In [0]:

WITH

base_table AS(
  SELECT * FROM imdb.raw.title_basics
  ),

-- Completeness
  null_check AS(
    SELECT * FROM base_table
    WHERE genres IS NULL
  ), -- no null values in tconst

-- Distinctiveness
  -- check duplicate genres within each merged string
    duplicate_check_genres_tconst AS(
      SELECT COUNT(*) 
      FROM 
        -- convert genres into array
        (SELECT *, split(trim(lower(genres)), ',') AS genres_array FROM base_table)
        -- if size(array_distinct) < size(array), then there are duplicates
        WHERE size(genres_array) <> size(array_distinct(genres_array))
    )   -- no duplicate genres in merged string

-- Uniformity 
  -- convert merged strings into arrays
  -- add 'Adult' to genres field for all isAdult = 1 entries (if not exist already)
  -- check character field relationships in below cell

SELECT * FROM duplicate_check_genres_tconst
